In [37]:
import pyvista as pv
import numpy as np
from scipy.interpolate import make_interp_spline
pv.set_jupyter_backend('client')
from generate_noisy_pipe import *

# Noisy cylinder

In [43]:
# Define mesh path
# case_path = '/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/mesh_test/'
case_path = '/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/NP_0042_bent/'
mesh_path = case_path + 'mesh/'
geom_path = case_path + 'geom/'

In [44]:
# Load geometry files
fluid = pv.read(geom_path + "fluid.vtp")
# solid = pv.read(geom_path + "solid.vtp")
# combined = pv.read(geom_path + "combined.vtp")
# regions = np.load(geom_path + "region.npy")
spline = np.load(geom_path + "spline.npy")
contours = np.load(geom_path + "contours.npy")
# Visualize
pl = pv.Plotter()
pl.add_mesh(fluid, cmap='tab10',show_edges=True)
# pl.add_mesh(solid, cmap='tab10',show_edges=True)
pl.add_points(spline, color='red')
for i in range(contours.shape[0]):
    pl.add_points(contours[i,:,:], color='blue')

pl.show()

Widget(value='<iframe src="http://localhost:36441/index.html?ui=P_0x7328c054dac0_27&reconnect=auto" class="pyv…

In [40]:
# Load fluid
f_vol = pv.read(mesh_path + "fluid/mesh-complete.vtu")
f_inlet = pv.read(mesh_path + "fluid/mesh-surfaces/inlet.vtp")
f_outlet = pv.read(mesh_path + "fluid/mesh-surfaces/outlet.vtp")
f_interface = pv.read(mesh_path + "fluid/mesh-surfaces/interface.vtp")
# f_vol_c = f_vol.clip(normal='y',origin=(0,0,0),invert=False)
# f_inlet = f_inlet.clip(normal='y',origin=(0,0,0),invert=False)
# f_outlet = f_outlet.clip(normal='y',origin=(0,0,0),invert=False)
# f_interface = f_interface.clip(normal='x',origin=(0,0,0),invert=False)

# Load solid
# s_vol = pv.read(mesh_path + "solid/mesh-complete.vtu")
# s_inlet = pv.read(mesh_path + "solid/mesh-surfaces/inlet.vtp")
# s_outlet = pv.read(mesh_path + "solid/mesh-surfaces/outlet.vtp")
# s_interface = pv.read(mesh_path + "solid/mesh-surfaces/interface.vtp")
# s_outside = pv.read(mesh_path + "solid/mesh-surfaces/outside.vtp")
# s_vol_c = s_vol.clip(normal='y',origin=(0,0,0),invert=False)
# s_inlet = s_inlet.clip(normal='y',origin=(0,0,0),invert=False)
# s_outlet = s_outlet.clip(normal='y',origin=(0,0,0),invert=False)
# s_interface = s_interface.clip(normal='y',origin=(0,0,0),invert=False)
# s_outside = s_outside.clip(normal='y',origin=(0,0,0),invert=False)


# Some camera stuff
cam = [(0,-10,0),(0,0,0),(0,0,1)]#[(13,3,3),(3,0,0),(0,0,1)]
cam2 = [(0,-25,0),(0,0,0),(0,0,1)]

plotter1 = pv.Plotter(shape=(1,2),off_screen=True)
# Volume mesh
plotter1.subplot(0,0)
plotter1.camera_position = cam
plotter1.add_mesh(f_vol, color='cyan', edge_opacity=0.5, show_edges=True) # Fluid
# plotter1.add_mesh(s_vol_c, color='magenta', edge_opacity=0.5, show_edges=True) # Solid
plotter1.add_text("Volume Mesh", font_size=12)
plotter1.add_axes()
# Surface meshes
plotter1.subplot(0,1)
plotter1.camera_position = cam
# Fluid
plotter1.add_mesh(f_inlet, color='cyan', opacity=1, edge_opacity=0.5, show_edges=True)
plotter1.add_mesh(f_outlet, color='magenta', opacity=1, edge_opacity=0.5, show_edges=True)
plotter1.add_mesh(f_interface, color='green', opacity=0.5, edge_opacity=0.5, show_edges=True)
# Solid
# plotter1.add_mesh(s_inlet, color='cyan', edge_opacity=0.5, show_edges=True)
# plotter1.add_mesh(s_outlet, color='magenta', edge_opacity=0.5, show_edges=True)
# plotter1.add_mesh(s_outside, color='red', edge_opacity=0.5, show_edges=True)
plotter1.add_text("Surface Mesh", font_size=12)
plotter1.add_axes()
plotter1.show()
# plotter1.save_graphic('mesh.eps')

plotter2 = pv.Plotter(shape=(1,2))
# Whole geometries
plotter2.subplot(0,0)
plotter2.camera_position = cam2
plotter2.add_mesh(f_vol_c, color='cyan', opacity=0.5, show_edges=False) # Fluid
# plotter2.add_mesh(s_vol_c, color='magenta', opacity=0.5, show_edges=False) # Solid
plotter2.add_axes()
plotter2.subplot(0,1)
plotter2.add_mesh(f_vol, color='cyan', opacity=0.5, show_edges=False) # Fluid
# plotter2.add_mesh(s_vol, color='magenta', opacity=0.5, show_edges=False) # Solid
plotter2.add_axes()
plotter2.show()
# plotter2.save_graphic('geom.eps')

Widget(value='<iframe src="http://localhost:36441/index.html?ui=P_0x7328bb7c3320_25&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36441/index.html?ui=P_0x7328b8d3b710_26&reconnect=auto" class="pyv…

In [41]:
import matplotlib.pyplot as plt 
# plt.hist(f_vol.compute_cell_sizes())
f = f_vol.compute_cell_sizes()
f = f.compute_cell_quality()
fig,ax=plt.subplots(1,2,figsize=(12,4))
ax[0].hist(f['Volume']**(1/3), bins=30)#, range=(0, 0.01))
ax[0].set_title('Cell Size')
ax[1].hist(f['CellQuality'], bins=30)#, range=(0, 0.01))
ax[1].set_title('Cell Quality')
delta_min = np.min(f['Volume'])**(1/3)
print('Approx. smallest cell volume: ' + str(np.min(f['Volume'])))
print('Approx. smallest cell length: ' + str(delta_min))
print('Approx. dt for CFL=1: ' + str(delta_min/100.)) 

Approx. smallest cell volume: 1.0036042644974348e-07
Approx. smallest cell length: 0.004647158651878327
Approx. dt for CFL=1: 4.6471586518783264e-05


In [42]:
f

UnstructuredGrid (0x73289821cee0)
  N Cells:    227592
  N Points:   47138
  X Bounds:   -5.872e+00, 5.790e+00
  Y Bounds:   -2.567e+00, 3.825e+00
  Z Bounds:   -2.154e+00, 1.779e+00
  N Arrays:   6